In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.regularizers import l1,l2,l1_l2

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
df = pd.read_csv('df_train.csv')
df

,5160399000.0,5160532000.0,5160665000.0,5160798000.0,5160931000.0,5161064000.0,5161197000.0,5161330000.0,5161463000.0,5161596000.0,...,5178354000.0,5178487000.0,5178620000.0,5178753000.0,5178886000.0,5179019000.0,5179152000.0,5179285000.0,5179418000.0,target
0,7.172340,13.318998,15.115269,12.120115,12.898385,13.807718,15.543046,14.583542,11.757287,12.031413,...,12.937673,12.112266,15.071740,15.653405,15.110099,14.973473,13.874439,15.763504,14.146375,0
1,8.772359,11.914471,14.203160,11.378137,11.834629,12.236748,14.267263,14.501087,11.912227,11.376563,...,11.545565,10.424321,13.245390,13.355729,12.802243,13.800442,12.812164,14.272315,12.451538,0
2,8.391178,11.156736,14.503078,12.438627,12.427057,13.169839,14.929343,14.835480,12.524612,12.274352,...,11.538292,10.504514,12.372451,12.029099,12.614220,13.682375,11.623414,13.791919,12.385667,0
3,6.163323,10.432897,15.272127,13.891915,12.910382,9.931044,12.755279,12.925886,12.226484,13.552130,...,11.433129,11.008790,13.047578,12.179321,12.295100,14.118652,12.313926,12.769601,11.562403,0
4,8.363369,11.315283,14.668475,13.853479,13.341522,10.920540,13.074407,12.922540,11.044026,13.024234,...,11.894255,11.838393,13.996025,13.175538,12.259200,13.906978,11.991285,12.603871,11.924631,0
5,8.240896,13.739551,16.909309,15.751151,14.918041,13.021907,15.742979,13.975338,11.833842,15.607986,...,12.337350,14.301595,16.914551,16.342760,15.981591,16.695617,13.586391,15.972772,15.946127,0
6,8.199476,14.391475,16.913791,14.984069,15.889930,14.243103,16.211843,14.473472,11.977827,15.824573,...,12.634924,14.355559,16.362179,16.782259,16.037976,16.848422,14.666279,16.549522,14.835597,0
7,8.853883,14.978434,16.108675,14.508603,16.282452,15.486989,16.696201,15.084880,12.343903,16.218308,...,14.137195,14.221792,16.397888,17.018057,15.751225,16.043181,13.503233,17.207232,14.548845,0
8,11.864605,14.443004,15.056640,13.854646,16.628577,15.871353,16.604690,14.021480,11.470074,14.144264,...,14.168295,14.729098,16.445488,17.023000,16.745621,16.771568,13.254674,16.521037,14.461398,0
9,12.044046,13.901843,14.996692,13.340502,15.901869,16.480884,16.409730,13.628276,11.467872,13.878743,...,13.211286,14.162416,16.526667,17.388181,16.718023,16.690297,13.601106,16.481902,13.965977,0


In [4]:
# df.loc[df['target'] <= 5, 'target'] = 0
# df.loc[(df['target'] <= 11) & (df['target'] > 5), 'target'] = 1
# df.loc[df['target'] > 11, 'target'] = 2

In [5]:
df = df.T
df

,0,1,2,3,4,5,6,7,8,9,...,15200,15201,15202,15203,15204,15205,15206,15207,15208,15209
5160399000.0,7.172340,8.772359,8.391178,6.163323,8.363369,8.240896,8.199476,8.853883,11.864605,12.044046,...,37.309982,37.557060,37.800302,37.716031,37.294341,36.892047,37.046907,36.866645,36.765690,37.269020
5160532000.0,13.318998,11.914471,11.156736,10.432897,11.315283,13.739551,14.391475,14.978434,14.443004,13.901843,...,38.692698,39.102889,38.711065,38.975965,38.949129,39.617401,39.946217,39.890295,39.919843,39.478383
5160665000.0,15.115269,14.203160,14.503078,15.272127,14.668475,16.909309,16.913791,16.108675,15.056640,14.996692,...,43.182949,43.569618,43.641231,43.512563,43.927987,44.132555,43.997264,43.831992,43.940200,43.491688
5160798000.0,12.120115,11.378137,12.438627,13.891915,13.853479,15.751151,14.984069,14.508603,13.854646,13.340502,...,41.123122,40.913973,41.410898,41.669700,41.915385,42.106085,42.155232,41.872157,41.998607,42.036139
5160931000.0,12.898385,11.834629,12.427057,12.910382,13.341522,14.918041,15.889930,16.282452,16.628577,15.901869,...,41.812858,41.910850,42.465279,42.182623,41.937857,41.621815,41.669003,41.812620,42.261537,42.405809
5161064000.0,13.807718,12.236748,13.169839,9.931044,10.920540,13.021907,14.243103,15.486989,15.871353,16.480884,...,42.416561,42.457891,42.955830,43.167262,43.252549,43.381185,43.198363,43.486500,43.118331,43.336053
5161197000.0,15.543046,14.267263,14.929343,12.755279,13.074407,15.742979,16.211843,16.696201,16.604690,16.409730,...,44.102796,43.933172,44.348377,44.461769,44.446746,44.507880,44.676234,44.155800,43.939217,43.879691
5161330000.0,14.583542,14.501087,14.835480,12.925886,12.922540,13.975338,14.473472,15.084880,14.021480,13.628276,...,42.775897,42.633823,43.154945,43.160982,43.097960,43.353797,43.521322,42.999448,42.678664,42.778234
5161463000.0,11.757287,11.912227,12.524612,12.226484,11.044026,11.833842,11.977827,12.343903,11.470074,11.467872,...,40.032608,40.567886,40.419354,40.284942,40.686545,40.906892,40.393558,40.373486,40.608265,40.296899
5161596000.0,12.031413,11.376563,12.274352,13.552130,13.024234,15.607986,15.824573,16.218308,14.144264,13.878743,...,40.274331,40.562422,40.376890,40.248417,40.340744,40.640464,40.557346,40.872037,41.014729,40.563734


In [6]:
sequences = list()
for i in range(df.shape[1]):
    values = df.iloc[:-1,i].values
    sequences.append(values)
targets = df.iloc[-1, :].values

In [7]:
len(sequences)

15210

In [8]:
sequences[0]

array([ 7.17234047, 13.31899781, 15.11526853, 12.12011535, 12.89838533,
       13.80771827, 15.5430457 , 14.58354249, 11.75728735, 12.03141335,
       12.77284521, 12.78153054, 11.09129892, 11.08719423, 10.32386674,
        9.97235726, 10.61110839,  8.92217133, 10.44102193, 11.81295697,
       14.60725906, 15.34749356, 15.29154604, 15.38382071, 17.12117606,
       16.99459822, 17.11184767, 15.89641411, 14.563588  , 12.88051489,
       14.13096786, 14.99422852, 12.39221563, 12.39080188, 10.19337126,
       10.75794911, 13.28910346, 12.5761107 , 12.5176845 , 14.38461933,
       12.72646239, 12.57594704, 14.99343339, 15.00171095, 16.04131217,
       16.06152557, 16.73092476, 17.69384976, 17.77854092, 17.82542784,
       17.07003689, 16.89298112, 16.78195861, 16.69825677, 16.77953904,
       16.20400008, 15.4376026 , 14.31727986, 13.06644462, 12.44507043,
       11.87834992,  9.6339479 ,  9.98000108, 10.6455822 , 10.99836535,
       11.24450944, 11.44267255, 12.73587437, 14.39489327, 16.92

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils
# labelencoder_X_1 = LabelEncoder()
# X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
# labelencoder_X_2 = LabelEncoder()
# X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(targets)
encoded_y = encoder.transform(targets)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
targets = dummy_y

In [11]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, y_train = sequences, targets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.08, random_state=1)

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals.joblib import dump, load
# sc = StandardScaler()
sc = load('std_scaler_smooth.bin')
X_train = sc.transform(X_train)
# X_test = sc.transform(X_test)
X_val = sc.transform(X_val)
# sc = MinMaxScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

X_train, X_val

/home/hongyu/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


(array([[ 0.01690185, -0.20545567, -0.24419498, ..., -0.22409581,
         -0.36910096, -0.31737886],
        [ 1.3384339 ,  0.53213175,  0.44717281, ...,  0.61856453,
          0.58925628,  0.62409537],
        [ 1.35589617,  1.38247085,  1.46816656, ...,  1.30835327,
          1.35014508,  1.42462305],
        ...,
        [-0.26756809,  0.16628436,  0.07638509, ...,  0.05777043,
          0.19842118,  0.07414391],
        [-0.71466385, -0.58890761, -0.74438187, ..., -0.73080378,
         -0.811521  , -0.78288819],
        [ 0.66072923,  0.6532634 ,  0.65688728, ...,  0.70261889,
          0.64700206,  0.71254075]]),
 array([[-0.02986961,  0.01183467,  0.09033338, ...,  0.02016512,
          0.0484155 ,  0.0589722 ],
        [ 0.47420165,  0.52526126,  0.58918153, ...,  0.47453451,
          0.5152193 ,  0.52026648],
        [-0.60141307, -0.60363401, -0.54388274, ..., -0.66594202,
         -0.63882365, -0.62722956],
        ...,
        [-0.67633589, -0.77502777, -0.77948015, ..., -

In [13]:
X_train.shape, X_val.shape

((13993, 144), (1217, 144))

In [14]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

In [15]:
X_train.shape, X_val.shape

((13993, 144, 1), (1217, 144, 1))

In [20]:
model = Sequential()
model.add(LSTM(144, return_sequences=True, input_shape=(144, 1), kernel_regularizer=l2(0.01)))
# , kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
# model.add(LSTM(128, dropout = 0.2, recurrent_dropout = 0.2, input_shape=(12, 1),return_sequences=False))
model.add(Dropout(0.3))
# model.add(LSTM(72, dropout = 0.2, return_sequences=False, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
# model.add(Dropout(0.1))
# model.add(LSTM(64))
# model.add(Dense(output_dim = 64, init = 'uniform', activation = 'relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(18, activation='softmax'))
# adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto')

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 144, 144)          84096     
_________________________________________________________________
dropout_2 (Dropout)          (None, 144, 144)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 20736)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2073700   
_________________________________________________________________
dense_3 (Dense)              (None, 18)                1818      
Total params: 2,159,614
Trainable params: 2,159,614
Non-trainable params: 0
_________________________________________________________________


In [24]:
import os
model_filename = 'model-{epoch:03d}-{acc:03f}-{val_acc:03f}-{val_loss:03f}.h5'
checkpoint_path = os.path.join('temp_lstm_smoothOnly/', model_filename)
monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 10, verbose = 1, mode = 'auto')
checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [25]:
# Fitting our model 
print('Train...')
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), callbacks = [monitor, checkpoint], verbose = 1, batch_size = 128, nb_epoch = 100)

Train...
Train on 13993 samples, validate on 1217 samples
Epoch 1/100
13993/13993 [==============================] - 18s 1ms/step - loss: 0.1491 - acc: 0.9541 - val_loss: 0.3500 - val_acc: 0.8809

Epoch 00001: val_loss improved from inf to 0.35002, saving model to temp_lstm_smoothOnly/model-001-0.954120-0.880855-0.350018.h5
Epoch 2/100
13993/13993 [==============================] - 18s 1ms/step - loss: 0.1260 - acc: 0.9606 - val_loss: 0.3560 - val_acc: 0.8825

Epoch 00002: val_loss did not improve
Epoch 3/100
13993/13993 [==============================] - 19s 1ms/step - loss: 0.1285 - acc: 0.9623 - val_loss: 0.2992 - val_acc: 0.9047

Epoch 00003: val_loss improved from 0.35002 to 0.29924, saving model to temp_lstm_smoothOnly/model-003-0.962267-0.904684-0.299244.h5
Epoch 4/100
13993/13993 [==============================] - 18s 1ms/step - loss: 0.1342 - acc: 0.9588 - val_loss: 0.3605 - val_acc: 0.8833

Epoch 00004: val_loss did not improve
Epoch 5/100
13993/13993 [=======================

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/LSTM_18pts_smoothOnly.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
# model.save_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding/weights_mixed/LSTM3sec_weight.h5")
print("Saved model to disk")

In [5]:
from keras.models import model_from_json
# load json and create model
json_file = open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/LSTM_18pts_smoothOnly.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
# model.load_weights("/home/hongyu/Documents/Spring2020/ECE_research/3_section/weights_mixed/LSTM3sec_weight.h5")
print("Loaded model from disk")

Loaded model from disk
